In [336]:
import numpy as np
from src.backend.state_estimation.config.vehicle_params import VehicleParams
from src.backend.state_estimation.measurments.measurement_transformation.steering_to_wheel_angle import measure_delta_wheel_angle

In [337]:
def moment_coefficients(steering: float):
    delta = measure_delta_wheel_angle(steering)
    lf = VehicleParams.lf
    a2 = VehicleParams.a / 2
    b2 = VehicleParams.b / 2
    return np.array([
        lf * np.sin(delta[0]) + a2 * np.cos(delta[0]), lf * np.sin(delta[1]) - a2 * np.cos(delta[1]), b2, -b2,
    ])

def get_torque_ref(Tcmd, Tmax):
    if Tcmd > sum(Tmax):
        return Tmax
    else:
        return Tcmd * Tmax / sum(Tmax)
    
def get_moment_allocation(Mz, Tmax, Mz_coefficients):
    inv_Tmax = 1 / Tmax
    A_raw = np.array([
        Mz_coefficients, 
        [inv_Tmax[0], 0, -inv_Tmax[2], 0],
        [0, inv_Tmax[1] , 0, -inv_Tmax[3]],
        [inv_Tmax[0], inv_Tmax[1], 0, 0],
        [0, 0, inv_Tmax[2], inv_Tmax[3]],
        [inv_Tmax[0], 0, 0, inv_Tmax[3]],
        [0, inv_Tmax[1], inv_Tmax[2], 0],
        np.ones(4),
    ],dtype=np.float64)
    B_raw = np.linalg.pinv(A_raw)
    torques = B_raw @ np.array([Mz, 0, 0, 0, 0, 0, 0, 0], dtype=np.float64)   
    return torques

def get_residual_torque_allocation(Mz, Tmax, Mz_coefficients):
    if Mz == 0:
        return np.zeros(4)
    
    inv_Tmax = 1 / Tmax
    if Mz_coefficients[0] == 0 and Mz_coefficients[2] == 0:
        A_raw = np.array([
            Mz_coefficients, 
            [0, inv_Tmax[1] , 0, -inv_Tmax[3]],
        ],dtype=np.float64)
    elif Mz_coefficients[1] == 0 and Mz_coefficients[3] == 0:
        A_raw = np.array([
            Mz_coefficients, 
            [inv_Tmax[0], 0, -inv_Tmax[2], 0],
        ],dtype=np.float64)
    else:
        raise ValueError(f"Not right or left residuals: {Mz_coefficients}")
    B_raw = np.linalg.pinv(A_raw)
    torques = B_raw @ np.array([Mz, 0], dtype=np.float64)   
    return torques

In [338]:
Tmax = np.array([100, 100, 200, 200])
Tcmd = 400
Mz_cmd = -100
steering = 0

In [339]:
T_refs = get_torque_ref(Tcmd, Tmax)
Mz_coefficients = moment_coefficients(steering)
current_Mz = Mz_coefficients @ T_refs
T_margins = Tmax - T_refs
print("Torque Maximum", Tmax)
print("Torque reference", T_refs.astype(int))
print("Current Moment", int(current_Mz))
print("Torque Margins", (T_margins).astype(int))

Torque Maximum [100 100 200 200]
Torque reference [ 66  66 133 133]
Current Moment 0
Torque Margins [33 33 66 66]


In [340]:
Mz_error = Mz_cmd - current_Mz
print("Moment error", int(Mz_error))

Moment error -100


In [341]:
Mz_allocation = get_moment_allocation(Mz_error, Tmax, Mz_coefficients)
print("Moment allocation", Mz_allocation.astype(int))

Moment allocation [-26  26 -53  53]


In [342]:
T_residuals = -np.minimum(T_margins - Mz_allocation, 0)
Mz_residuals = Mz_coefficients @ T_residuals
print("Residuals", T_residuals.astype(int), int(Mz_residuals))

Residuals [0 0 0 0] 0


In [343]:
Mz_residual_coefficients = (T_residuals == 0).astype(int) * Mz_coefficients
print("Mz_residual_coefficients", Mz_residual_coefficients)
Mz_residual_allocation = get_moment_allocation(Mz_residuals, Tmax, Mz_residual_coefficients)
print("Mz_residual_allocation", Mz_residual_allocation.astype(int))
Mz_residual_allocation *= (T_residuals == 0).astype(int)
print("Mz_residual_allocation", Mz_residual_allocation.astype(int))

Mz_residual_coefficients [ 0.62051896 -0.62051896  0.62       -0.62      ]
Mz_residual_allocation [0 0 0 0]
Mz_residual_allocation [0 0 0 0]


In [344]:
T_final = T_refs + Mz_allocation + Mz_residual_allocation
T_final = np.minimum(T_final, Tmax)
print("Final Torque allocation", (T_final + 0.5).astype(int))
print("Total T: ", Tcmd, "-->", int(sum(T_final) + 0.5))
print("Total Mz:", Mz_cmd, "-->", int(T_final @ Mz_coefficients + 0.5))
print("Grip Allocation", np.round(T_final / Tmax, 2))

Final Torque allocation [ 40  94  80 187]
Total T:  400 --> 400
Total Mz: -100 --> -99
Grip Allocation [0.4  0.94 0.4  0.94]
